In [3]:
import pandas as pd
from pandas import DataFrame
import os
import json

prefix_old: str = 'old'
prefix_new: str = 'new'
data_folder: str = '../../data'
all_new_data_files: list = [file_name for file_name in os.listdir(data_folder) if prefix_new in file_name]
all_old_data_files: list = [file_name for file_name in os.listdir(data_folder) if prefix_old in file_name]

In [4]:
# The list of the top 10 models from benchmark version 0.9-1.0
top_10_models: list = [
    'gpt-4-0613-t0.0--gpt-4-0613-t0.0',
    'claude-v1.3-t0.0--claude-v1.3-t0.0',
    'gpt-4-1106-preview-t0.0--gpt-4-1106-preview-t0.0',
    'gpt-4-t0.0--gpt-4-t0.0',
    'gpt-4-0314-t0.0--gpt-4-0314-t0.0',
    'claude-2.1-t0.0--claude-2.1-t0.0',
    'gpt-4-t0.0--gpt-3.5-turbo-t0.0',
    'claude-2-t0.0--claude-2-t0.0',
    'gpt-3.5-turbo-1106-t0.0--gpt-3.5-turbo-1106-t0.0',
    'gpt-3.5-turbo-0613-t0.0--gpt-3.5-turbo-0613-t0.0',   
]

In [9]:
data_old: DataFrame = None

for data_file in all_old_data_files:
    df: DataFrame = pd.read_json(data_folder + f'/{data_file}', lines=True)
    
    if data_old is None:
        data_old = df
    else:
        data_old = pd.concat([data_old, df])

In [ ]:
# Check the spelling of the top 10 models
for top_k_model in top_10_models:
    assert top_k_model in list(data_old.model.unique())

In [21]:
data_top_k: DataFrame = data_old[data_old.model.isin(top_10_models)]
data_successful: DataFrame = data_top_k[data_top_k.Success == 1]

In [25]:
from datasets import load_dataset, Dataset

In [15]:
for top_k_model in top_10_models:
    assert top_k_model in list(data_old.model.unique())

In [26]:
data: Dataset = Dataset.from_pandas(data_successful)

In [29]:
chat = data_successful.chat.iloc[1000]

In [52]:
def prepare_qu_pari_wise_data(_data: DataFrame) -> DataFrame:
    result_data: dict = {key: [] for key in _data.columns}
    
    for i, row in _data.iterrows():
        chat = row['chat']
        
        # there is only one qa pair
        if len(chat) == 2:
            for key in _data.columns:
                result_data[key].append(row[key])
        
        else:
            for i in range(2, len(chat), 2):
                for key in _data.columns:
                    if key != 'chat':
                        result_data[key].append(row[key])
                    else:
                        result_data[key].append(chat[:i])
            else:
                # catch the last chat pair
                for key in _data.columns:
                    result_data[key].append(row[key])
                        
    return result_data
    

In [53]:
result = prepare_qu_pari_wise_data(data_successful)

In [54]:
data_collapsed = pd.DataFrame(result)

In [41]:
data_collapsed.head()

,game,game_id,model,experiment,episode,Aborted,Lose,Success,player,chat
0,wordle_withclue,2,claude-v1.3-t0.0--claude-v1.3-t0.0,0_high_frequency_words_clue_no_critic,episode_1,0,0,1,Player1,"[{'role': 'user', 'content': 'You are a langua..."
1,wordle_withclue,6,claude-v1.3-t0.0--claude-v1.3-t0.0,0_high_frequency_words_clue_no_critic,episode_5,0,0,1,Player1,"[{'role': 'user', 'content': 'You are a langua..."
2,wordle_withclue,7,claude-v1.3-t0.0--claude-v1.3-t0.0,0_high_frequency_words_clue_no_critic,episode_6,0,0,1,Player1,"[{'role': 'user', 'content': 'You are a langua..."
3,wordle_withclue,8,claude-v1.3-t0.0--claude-v1.3-t0.0,0_high_frequency_words_clue_no_critic,episode_7,0,0,1,Player1,"[{'role': 'user', 'content': 'You are a langua..."
4,wordle_withclue,8,claude-v1.3-t0.0--claude-v1.3-t0.0,0_high_frequency_words_clue_no_critic,episode_7,0,0,1,Player1,"[{'role': 'user', 'content': 'You are a langua..."


In [55]:
data_collapsed.chat.iloc[3]

[{'role': 'user',
  'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\n To help you make an informed guess, you will receive a clue for the word, such as\nclue: snowy white.\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.\n\nFor example, the feedback for "apple" mi

In [56]:
data_collapsed.chat.iloc[4]

[{'role': 'user',
  'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\n To help you make an informed guess, you will receive a clue for the word, such as\nclue: snowy white.\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.\n\nFor example, the feedback for "apple" mi

In [57]:
data_collapsed.chat.iloc[5]

[{'role': 'user',
  'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\n To help you make an informed guess, you will receive a clue for the word, such as\nclue: snowy white.\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.\n\nFor example, the feedback for "apple" mi

In [59]:
data_collapsed.chat.iloc[6]

[{'role': 'user',
  'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\n To help you make an informed guess, you will receive a clue for the word, such as\nclue: snowy white.\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.\n\nFor example, the feedback for "apple" mi

In [58]:
data_successful.chat.iloc[3]

[{'role': 'user',
  'content': 'You are a language wizard who likes to guess words by using the given rules.\n\nWelcome to Wordle! You have six attempts to guess the target word, a valid English word of five lowercase letters (a-z). Please use the tags "guess:" and "explanation:" to provide a concise explanation for each guess.\n\n To help you make an informed guess, you will receive a clue for the word, such as\nclue: snowy white.\n\nHere is an example guess based on the clue:\nguess: apple\nexplanation: In the fairy tail Snow White, the girl is killed because she eats a poisoned apple. And the word apple has 5 letters.\n\nAfter each guess, your answer will be validated, and you will receive feedback indicating which letters are correct (green), which letters are correct but in the wrong position (yellow), and which letters are incorrect (red). This feedback can be useful in determining which letters to include or exclude in your next guess.\n\nFor example, the feedback for "apple" mi

In [63]:
data_successful.to_csv('../../data/clem_top_10_models_data_all_successful_episodes_no_preprocessing.csv')

In [64]:
data_collapsed.to_csv('../../data/clem_top_10_models_data_all_successful_episodes_individual_turns.csv')

In [65]:
data_old.columns

Index(['game', 'game_id', 'model', 'experiment', 'episode', 'Aborted', 'Lose',
       'Success', 'player', 'chat'],
      dtype='object')